<a href="https://colab.research.google.com/github/birat1/xAI-medical-diagnosis/blob/master/xAI_medical_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# 1. Install uv
!curl -LsSf https://astral.sh/uv/install.sh | sh
import sys
sys.path.append("/root/.local/bin")

# 2. Install SWI-Prolog system dependencies
!sudo apt-add-repository ppa:swi-prolog/stable -y
!sudo apt-get update
!sudo apt-get install swi-prolog -y

# 3. Verify SWI-Prolog is installed
!swipl --version

downloading uv 0.10.4 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/swi-prolog/stable/ubuntu/ jammy main'
Description:
Comprehensive Prolog implementation with extensive libraries and development tools.   Primarily targetted at teaching, RDF processing and web-related tasks, such as creating web services or analysing web content.

Official PPAs for SWI-Prolog. See https://www.swi-prolog.org for further information.
More info: https://launchpad.net/~swi-prolog/+archive/ubuntu/stable
Adding repository.
Found existing deb entry in /etc/apt/sources.list.d/swi-prolog-ubuntu-stable-jammy.list
Adding deb entry to /etc/apt/sources.list.d/swi-prolog-ubuntu-stable-jammy.list
Found existing deb-src entry in /etc/apt/sources.list.d/swi-prolog-ubuntu-stable-jammy.list
Adding disabled deb-src entry to /etc/apt/sources

In [18]:
# Check GPU is available (CUDA)
!nvidia-smi

Thu Feb 19 15:08:23 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [22]:
# Clone the repository
!git clone https://github.com/birat1/xAI-medical-diagnosis.git
%cd xAI-medical-diagnosis
!uv pip install --system -r pyproject.toml

%cd src

Cloning into 'xAI-medical-diagnosis'...
remote: Enumerating objects: 590, done.
remote: Counting objects: 100% (590/590), done.
remote: Compressing objects: 100% (399/399), done.
remote: Total 590 (delta 259), reused 509 (delta 181), pack-reused 0 (from 0)
Receiving objects: 100% (590/590), 10.99 MiB | 19.71 MiB/s, done.
Resolving deltas: 100% (259/259), done.
/content/xAI-medical-diagnosis/xAI-medical-diagnosis/src/xAI-medical-diagnosis/xAI-medical-diagnosis/src/xAI-medical-diagnosis/src/xAI-medical-diagnosis
Using Python 3.12.12 environment at: /usr
Resolved 94 packages in 443ms
Audited 94 packages in 1ms
/content/xAI-medical-diagnosis/xAI-medical-diagnosis/src/xAI-medical-diagnosis/xAI-medical-diagnosis/src/xAI-medical-diagnosis/src/xAI-medical-diagnosis/src


In [23]:
# Data Preprocessing
!python preprocessing.py --input ../data/raw/diabetes.csv --output ../data/processed

Data loaded successfully from ../data/raw/diabetes.csv
Correlation heatmap saved to ../data/processed/correlation_heatmap.png
Imputed symbolic data saved to ../data/symbolic
Completed preprocessing with IterativeImputer and SMOTE.
Train size: 1684 | Val size: 320 | Test size: 400
Processed data saved to ../data/processed
Preprocessing artifacts saved to ../data/processed/preprocess_artifacts.pkl


In [24]:
# Model Training
!python models.py

Global seed set to 42
Starting Random Forest training with 5-Fold CV...
Fitting 5 folds for each of 162 candidates, totalling 810 fits
[CV] END class_weight=balanced, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END class_weight=balanced, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END class_weight=balanced, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END class_weight=balanced, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END class_weight=balanced, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END class_weight=balanced, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.9s
[CV] END class_weight=balanced, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; to

In [25]:
# Optimise Thresholds
!python optimise_threshold.py


--- Random Forest Optimisation (VALIDATION) ---
AUPRC: 0.9664
Optimal Threshold: 0.6300
Max F1-Score at this threshold: 0.9189

--- Decision Tree Optimisation (VALIDATION) ---
AUPRC: 0.8514
Optimal Threshold: 0.3333
Max F1-Score at this threshold: 0.8305

--- Multi-Layer Perceptron Optimisation (VALIDATION) ---
AUPRC: 0.7666
Optimal Threshold: 0.3194
Max F1-Score at this threshold: 0.7092

Optimal thresholds saved to ../results/metrics/thresholds.json

Plot saved to ../results/metrics/optimisation_results.png
--------------------------------------------------

--- Random Forest (Test) Final Evaluation (threshold 0.6300) ---
F1-Score: 0.9420
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       263
           1       0.94      0.95      0.94       137

    accuracy                           0.96       400
   macro avg       0.95      0.96      0.96       400
weighted avg       0.96      0.96      0.96       400


--- Decision Tree (Tes

In [26]:
# Evaluate models
!python evaluate_models.py

Test data loaded successfully.

Random Forest (Test) Evaluation (threshold=0.6300) ---
F1-Score: 0.9420 | AUPRC: 0.9840
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       263
           1       0.94      0.95      0.94       137

    accuracy                           0.96       400
   macro avg       0.95      0.96      0.96       400
weighted avg       0.96      0.96      0.96       400


Decision Tree (Test) Evaluation (threshold=0.3333) ---
F1-Score: 0.8467 | AUPRC: 0.9067
              precision    recall  f1-score   support

           0       0.96      0.86      0.91       263
           1       0.78      0.93      0.85       137

    accuracy                           0.89       400
   macro avg       0.87      0.90      0.88       400
weighted avg       0.90      0.89      0.89       400


Multi-Layer Perceptron (Test) Evaluation (threshold=0.3194) ---
F1-Score: 0.7492 | AUPRC: 0.7380
              precision    recall  f1-s

In [27]:
# Interpret
!python interpretability.py

Global seed set to 42
Generating SHAP explanations for Random Forest...
Generating LIME explanations for Random Forest...
Generating SHAP explanations for Decision Tree...
Generating LIME explanations for Decision Tree...
Generating SHAP explanations for Multi-Layer Perceptron...
Generating LIME explanations for Multi-Layer Perceptron...
Interpretability analysis completed for patients: [0, 1, 2, 3, 4].


In [28]:
# Explain
!python explainability.py

Global seed set to 42
Generating PyGol symbolic logical rules...
100% 842/842 [00:03<00:00, 253.51it/s]
100% 438/438 [00:01<00:00, 252.60it/s]
100% 631/631 [00:30<00:00, 20.88it/s]
100% 328/328 [00:14<00:00, 22.40it/s]
+----------+ Hypothesis +----------+
['target(A):-diabetespedigreefunction(A,g0),skinthickness(A,g0)', 'target(A):-age(A,g2),bmi(A,g0),diabetespedigreefunction(A,g3),pregnancies(A,g2)', 'target(A):-bmi(A,g4),diabetespedigreefunction(A,g0),insulin(A,g0)', 'target(A):-bloodpressure(A,g3),diabetespedigreefunction(A,g1),insulin(A,g1),skinthickness(A,g0)', 'target(A):-bloodpressure(A,g4),diabetespedigreefunction(A,g1),glucose(A,g0),pregnancies(A,g2)', 'target(A):-age(A,g4),bmi(A,g1),diabetespedigreefunction(A,g2),pregnancies(A,g1)', 'target(A):-glucose(A,g0),pregnancies(A,g3),skinthickness(A,g4)', 'target(A):-bloodpressure(A,g2),bmi(A,g4),diabetespedigreefunction(A,g4),insulin(A,g3)', 'target(A):-age(A,g4),bmi(A,g0),diabetespedigreefunction(A,g2),skinthickness(A,g1)', 'target